# AI Agents with LangChain

O que são agentes de IA?

Um agente de IA é um sistema que decide e executa ações para atingir objetivos, interagindo com usuários, dados e APIs. Combina modelos de linguagem, lógica de controle e ferramentas especializadas para resolver tarefas de forma autônoma.

**Principais características:**
- **Autonomia:** Seleciona passos sem intervenção constante.  
- **Ferramentas (tools):** Chama funções externas (transcrição, busca, BD).  
- **Contexto:** Mantém histórico para respostas coerentes.  
- **Fluxos (chains):** Encadeia prompts e processa saídas de LLMs.  
- **RAG:** Recupera dados externos antes de gerar respostas.

**Benefícios:**
- Automatiza resumos, buscas e relatórios.  
- Integra serviços e APIs num único fluxo.  
- Escala atendimentos e análises.  
- Facilita criação de chatbots e assistentes virtuais.

Em essência, funciona como um orquestrador inteligente que une modelos de linguagem e ferramentas para pipelines dinâmicos e contextuais.

O que nosso agente fará?

<img src="docs/agent.png" alt="agent" style="width:800px;"/>

## 1. Objetivos

- Mostrar como configurar o ambiente e dependências.
- Entender o uso de variáveis de ambiente (.env).
- Explorar como definimos ferramentas com @tool.
- Ver como montamos um agente com LangChain.
- Executar o agente em um exemplo prático.

---

## 2. Configuração do Ambiente

### 2.1 Criando e ativando o venv

In [ ]:
# Criando venv
python3 -m venv venv

In [ ]:
# Ativando venv

# No Linux/Mac
source venv/bin/activate

# No Windows (PowerShell)
venv\Scripts\Activate.ps1

### 2.2 Instalando dependências

In [ ]:
pip install -r requirements.txt

---

## 3. Variáveis de Ambiente (.env)
Usamos python-dotenv para carregar configurações sem expor senhas ou chaves no código.

In [ ]:
from dotenv import load_dotenv
load_dotenv()  # Lê o arquivo .env e define variáveis em os.environ

CONVERSATION_MODEL: escolhe o modelo de LLM (ex.: llama2 ou openai).

---

## 4. Importações Principais

In [ ]:
import sys, os, logging
from typing import List, Optional
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_community.tools import tool
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain.schema.runnable import RunnableLambda

# Ajuste de path para módulos locais
sys.path.append(os.path.dirname(os.path.abspath(__file__)))

# Módulos customizados
from transcribe import transcribe_youtube_videos
from news import search_news
from llm import LLM

### Principais ferramentas usadas:

**[Langchain](https://www.langchain.com/)**

LangChain é um framework open‑source para criar aplicações com modelos de linguagem. Oferece abstrações para LLMs, gerenciamento de prompts, cadeias de processamento, agentes, integração com bases de dados e memória, permitindo desenvolver chatbots, sistemas RAG e fluxos complexos de IA de forma modular.

**[Ollama](https://ollama.com/)**

Ollama é uma plataforma CLI e servidor local para hospedar e executar modelos de linguagem (LLMs). Permite baixar, gerenciar e servir LLMs no próprio hardware, oferecendo API REST e integração com frameworks como LangChain para geração de texto, embeddings e pipelines de RAG sem depender de serviços em nuvem.

**LLM usadas**

- llama3.1:latest
- gpt-3.5-turbo

---

## 5. Definindo Ferramentas (@tool)

Cada função marcada com `@tool` vira uma ferramenta que o agente pode chamar.
Cada ferramenta deve ter uma **descrição muito bem detalhada** para o agente saber quando e como deve invocar ela.

As ferramentas podem e devem executar o que for preciso para concluir seu trabalho, como por exemplo:
- Chamar uma API Http
- Chamar outra LLM
- Salvar arquivos
- ETC

### 5.1 Transcrição de Vídeos

In [ ]:
@tool
def transcribe_tool(url: str) -> str:
    """
    DESCRIPTION HERE...
    """
    # Recebe URL de YouTube, retorna texto transcrito
    transcription = transcribe_youtube_videos(url, FILES_FOLDER_PATH)
    return transcription or "I don't know"

### 5.2 Resumo de Transcrição

In [ ]:
@tool
def summarize_tool(transcription: str) -> str:
    """
    DESCRIPTION HERE...
    """
    # Usa LLM para resumir em bullet points
    llm = LLM.load_ollama_model(model_name, temperature=0.5)
    messages = [SystemMessage(...), HumanMessage(transcription)]
    return llm.invoke(messages)

### 5.3 Busca de Notícias

In [ ]:
@tool
def search_news_tools(text: str, max_results: Optional[int] = None) -> List[dict]:
    """
    DESCRIPTION HERE...
    """
    # Extrai keywords via LLM e busca em API de notícias
    keywords = llm.invoke([...])
    return search_news(query=keywords, max_results=max_results)

### 5.4 Salvando Resultado

In [ ]:
@tool
def save_tool(response: str) -> None:
    """
    DESCRIPTION HERE...
    """
    with open("agent_output.md", "w") as f:
        f.write(response)

---

## 6. Montando o Agente

1. Criar a caixa de ferramentas

In [ ]:
toolkit = [transcribe_tool, summarize_tool, search_news_tools, save_tool]

2. Prompt Template: define a conversa inicial.

In [ ]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Você é um assistente de IA..."),
    MessagesPlaceholder("history", optional=True),
    ("human", "{input}"),
    MessagesPlaceholder("agent_scratchpad"),
])

3. Criando o agente:

In [ ]:
agent_llm = LLM.load_open_ai_model(temperature=0)
agent = create_openai_functions_agent(agent_llm, toolkit, prompt)
executor = AgentExecutor(agent=agent, tools=toolkit, verbose=True)

- verbose=True: imprime passo a passo da decisão do agente.

---

## 7. Exemplo de Execução

In [ ]:
def run_agent():
    text_input = [
        "Summarize the video at https://...",
        "Search for 5 news articles...",
        "Save the summary, keywords and news in pt-BR.",
    ]
    result = executor.invoke({"input": text_input})
    print(result["output"])

- O agente vai chamar, em ordem, transcribe_tool, summarize_tool, search_news_tools e save_tool.

---

## 8. Fluxo de Dados

1. Entrada: URLs e instruções do usuário.
2. Ferramentas: transcrição → resumo → busca → gravação.
3. Agente: coordena chamadas, decide qual ferramenta usar.
4. Saída: arquivo agent_output.md com o resultado final.

---

## 9. Conclusão

- Agentes de IA permitem combinar várias capacidades (tools) de forma dinâmica.
- O LangChain simplifica registro de ferramentas e roteirização de prompts.
- Com poucos trechos de código, criamos um pipeline de RAG (Retrieval-Augmented Generation).

> Próximos passos: experimentar outros modelos, criar novas ferramentas e ajustar prompts para melhorar resultados.
